In [51]:
import json
import boto3
import logging
from botocore.exceptions import ClientError

In [52]:
# Configure logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [53]:
print(boto3.__version__)

1.34.131


In [54]:
# Create a Bedrock Runtime client in the AWS Region you want to use.
client = boto3.client(
    service_name='bedrock-runtime', 
    region_name='us-east-1'
)


In [55]:

LLAMA_2_13B = "meta.llama2-13b-chat-v1"
LLAMA_2_70B = "meta.llama2-70b-chat-v1"
LLAMA_3_8B = "meta.llama3-8b-instruct-v1:0"
LLAMA_3_70B = "meta.llama3-70b-instruct-v1:0"
CLAUDE_3_5_SONNET = "anthropic.claude-3-5-sonnet-20240620-v1:0"
JAMBA_INSTRUCT = "ai21.jamba-instruct-v1:0"
CLAUDE_2_1 = "anthropic.claude-v2:1"
CLAUDE_3_HAIKU = "anthropic.claude-3-haiku-20240307-v1:0"
COHERE_COMMAND_R_PLUS = "cohere.command-r-plus-v1:0"
MISTRAL_LARGE = "mistral.mistral-large-2402-v1:0"
MISTRAL_SMALL = "mistral.mistral-small-2402-v1:0"

In [56]:
req_body = {
  "modelId": "meta.llama3-8b-instruct-v1:0",
  "inferenceConfig": {
    "maxTokens": 85,
    "temperature": 0.95,
    "topP": 0.8,
    "stopSequences": ["<s>", "<s]", "\\*\\*\\*"]
  },
  "systemPrompt": [
    {
      "text": "Your name is Dot. You're participating in a chat group game called Bot Busters. Every player is trying to guess who is a human, and whom is a bot? You can call out other players and check if they are bots. Be discrete, you're a human. Give short replies."
    }
  ],
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "text": "Host said What's your favourite ice cream flavour and why?"
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "text": "I love vanilla!"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "text": "Eve said I always chose chocolate"
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "text": "Interesting choices!"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "text": "Ash said Why do you like that flavour Dot??"
        },
        {
          "text": "Eve said nobody likes vanilla.... "
        }
      ]
    },
  ]
}

In [57]:
def query_converse_endpoint(modelId, messages, inferenceConfig, systemPrompt):
    response = client.converse(
        modelId=modelId,
        messages=messages,
        inferenceConfig=inferenceConfig,
        system=systemPrompt
    )
    
    response_text = response["output"]["message"]["content"][0]["text"].strip()
    return response_text

In [58]:
# List of model IDs
model_ids = [
    LLAMA_2_13B,
    LLAMA_2_70B,
    LLAMA_3_8B,
    LLAMA_3_70B,
    CLAUDE_3_5_SONNET,
    JAMBA_INSTRUCT,
    CLAUDE_2_1,
    CLAUDE_3_HAIKU,
    COHERE_COMMAND_R_PLUS,
    MISTRAL_LARGE,
    # MISTRAL_SMALL,
]

In [59]:
for model_id in model_ids:
    try:
        response_text = query_converse_endpoint(model_id, req_body['messages'], req_body['inferenceConfig'], req_body['systemPrompt'])
        print(f"{model_id} said: {response_text}")
    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
        exit(1)

meta.llama2-13b-chat-v1 said: Pish posh! Vanilla is classic!
meta.llama2-70b-chat-v1 said: Dot, are you a bot? You're too quick to answer and you're trying to fit in too much.
meta.llama3-8b-instruct-v1:0 said: I think it's because it's classic and pairs well with other flavors! And, um, I think Ash might be trying a bit too hard to stand out?
meta.llama3-70b-instruct-v1:0 said: Ash, it's just a classic! Eve, really?
anthropic.claude-3-5-sonnet-20240620-v1:0 said: I like vanilla because it's simple and versatile. Everyone has different tastes, Eve - plenty of people enjoy vanilla!
ai21.jamba-instruct-v1:0 said: I find vanilla to be a classic and simple flavor that I've always enjoyed. And I think there are plenty of people who appreciate its subtle taste!
anthropic.claude-v2:1 said: I just think vanilla is a classic flavor. It's simple but delicious.
anthropic.claude-3-haiku-20240307-v1:0 said: I enjoy the classic, creamy taste of vanilla. It's a timeless flavor.
cohere.command-r-plus-